In [1]:
!pip install umap
!pip install trimap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=e0f0cdb49a50521b19906708a96c45fc90b5eb54bcbd64c781f18aa9270e9b29
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 646 kB 7.2 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391586 sha256=5fc280858051f9309f98b10d637b8908eb8531930f74885cb4466c1934cfba51
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import sys
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code')
sys.path.insert(1, '/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master')

In [6]:
## Importing libraries

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import math
import numpy as np
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.models.glyphs import Text
from bokeh.layouts import row
from bokeh.io import export_png
output_notebook()

## Importing local Python files

sys.path.append('../') 
from src import Preprocessing, Distances, Explanation, DR_algorithms, genericMethods
from src.LAPS_tabular import LapsExplainer

## Defining input paths (Update the Data_path attribute accordingly)

Data_path = "/content/drive/MyDrive/STU_Rafae_Baez_Ramirez/New_SNE_Code/LAPS_and_GAPS-master/data/"

### Step 1: Load and Pre-process Dataset

#### a. Load Dataset

In [65]:
df = pd.read_csv(Data_path+'breast_cancer.csv')
df = df.fillna(0)
#df = df.sample(100)
df.reset_index(inplace= True, drop= True)
df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


#### b. Identify Intrinsic Dimensionality

In [8]:
## Identify intrinsic dimensionality using: the Maximum likelihood intrinsic dimensionality estimator

int_dim = Preprocessing.repeated(Preprocessing.intrinsic_dim_scale_interval, 
                             df.values, 
                             mode='bootstrap', 
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1, 
                             k1=10, k2=20)
print ("Intrinsic dimensionality:", int_dim)

100%|██████████| 500/500 [00:07<00:00, 66.93it/s]

Intrinsic dimensionality: 1


#### c. Set Features and Target

In [66]:
model_features, target = Preprocessing.set_features_and_target(df)

y = (df[target].values.reshape(-1, ))
X_df = pd.DataFrame(df, columns=model_features)
#data_instance_number = 70
#POI = np.random.randint(0, X_df.shape[0], 100)

Features:  ['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave points_worst', 'symmetry_worst', 'fractal_dimension_worst']
Target:  ['diagnosis']


#### d. Identify Categorical Features

In [67]:
X_transformed, categorical_features, numeric_features, categorical_names = Preprocessing.identify_and_transform_features(df, model_features)

### Step 2: Run Dimensionality Reduction

In [68]:
df = pd.read_csv(Data_path+'breast_cancer.csv')
df = df.fillna(0)
df.reset_index(inplace= True, drop= True)
orig = df.values
df = pd.read_csv(Data_path+'BCancer.csv')
df = df.fillna(0)
df.reset_index(inplace= True, drop= True)
select = df.values

POI = []
print(orig)
for a in select:
  POI.append(np.where(np.all(orig == a, axis =1))[0][0])
print(POI)
print(len(POI))
print(POI)
ld_embedding = DR_algorithms.run_DR_Algorithm("tSNE", X_transformed)
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
divs = []
for data_instance_number in POI:  
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding, data_instance_number, X_df.values[data_instance_number], nbrs=10, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")
  divs.append(divergence)
  print(divergence)
print(np.mean(divs))

[[1.799e+01 1.038e+01 1.228e+02 ... 4.601e-01 1.189e-01 1.000e+00]
 [2.057e+01 1.777e+01 1.329e+02 ... 2.750e-01 8.902e-02 1.000e+00]
 [1.969e+01 2.125e+01 1.300e+02 ... 3.613e-01 8.758e-02 1.000e+00]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 2.218e-01 7.820e-02 1.000e+00]
 [2.060e+01 2.933e+01 1.401e+02 ... 4.087e-01 1.240e-01 1.000e+00]
 [7.760e+00 2.454e+01 4.792e+01 ... 2.871e-01 7.039e-02 0.000e+00]]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
99
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70

In [69]:
print(np.mean(divs))
print(np.min(divs))

0.6525993412086358
0.49322006919119576


[[1.799e+01 1.038e+01 1.228e+02 ... 4.601e-01 1.189e-01 1.000e+00]
 [2.057e+01 1.777e+01 1.329e+02 ... 2.750e-01 8.902e-02 1.000e+00]
 [1.969e+01 2.125e+01 1.300e+02 ... 3.613e-01 8.758e-02 1.000e+00]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 2.218e-01 7.820e-02 1.000e+00]
 [2.060e+01 2.933e+01 1.401e+02 ... 4.087e-01 1.240e-01 1.000e+00]
 [7.760e+00 2.454e+01 4.792e+01 ... 2.871e-01 7.039e-02 0.000e+00]]
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128]
99


In [ ]:
all_runs = 5
iterations = 300
#data_instance_number = 24

In [ ]:
import my_SNE


oldSNE = my_SNE.My_SNE(X_transformed.T, max_iterations=200)
results_orig_SNE = []

ld_embedding_oSNE = []
for _ in range(5):
  x_embed_SNE_old = oldSNE.fit_transform()
  ld_embedding_oSNE = np.array(x_embed_SNE_old[0]).T

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_oSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_orig_SNE.append([components[0], components[1], components[2], divergence])
  #print("Discrepancy in Feature Influence:", components[0])
  #print("Discrepancy in Neighborhood Content:", components[1])
  #print("Discrepancy in Neighborhood Order:", components[2])
  #print("Overall Divergence: ", divergence)
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_oSNE[:,0],
            y=ld_embedding_oSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

In [ ]:
np.save('results_old_50_500', np.array(results_orig_SNE, dtype=np.float32), allow_pickle=True)

In [ ]:
import new_SNE_torch
nSNE = new_SNE_torch.TorchTSNE(verbose=True, n_iter=200)
results_torch_SNE = []

for _ in range(5):
  ld_embedding_torch_SNE = nSNE.fit_transform(X_transformed)

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_torch_SNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_torch_SNE.append([components[0], components[1], components[2], divergence])
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_torch_SNE[:,0],
            y=ld_embedding_torch_SNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

In [ ]:
import TorchSNE
SNE = TorchSNE.TorchSNE(verbose=True, n_iter=200)
results_TorchSNE = []

for _ in range(5):
  ld_embedding_torch_SNE = SNE.fit_transform(X_transformed)

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_torch_SNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_TorchSNE.append([components[0], components[1], components[2], divergence])
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_torch_SNE[:,0],
            y=ld_embedding_torch_SNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

In [ ]:
import fSNE
import time
POI = [24, 86]
weights = [1.25,1.125,0.85]
fSNE_slow_inc = fSNE.fSNE(X_transformed, max_iter = 200, POI = POI, lambdas = weights)
start = time.time()
embedding_fnew = fSNE_slow_inc.fit_transform()
print(f'That took {time.time()-start} seconds')
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, embedding_fnew, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=embedding_fnew[:,0],
            y=embedding_fnew[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
import my_fSNE
fSNE_slow = my_fSNE.my_fSNE(X_transformed, max_iter=200, POI = POI, lambdas = weights)
start = time.time()
embedding_fold = fSNE_slow.fit_transform()
print(f'That took {time.time()-start} seconds')
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, embedding_fold, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=embedding_fold[:,0],
            y=embedding_fold[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
import my_SNE
normal_SNE = my_SNE.My_SNE(X_transformed.T, max_iterations=200)
start = time.time()
embedding, _ = normal_SNE.fit_transform()
embedding_oold = embedding.T
print(f'That took {time.time()-start} seconds')
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, embedding_oold, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=embedding_oold[:,0],
            y=embedding_oold[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
import TorchSNE
import torch
SNE = TorchSNE.TorchSNE(verbose=True, n_iter=iterations)
start = time.time()
embedding_onew = SNE.fit_transform(X_transformed)
print(f'That took {time.time()-start} seconds')
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, embedding_onew, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=embedding_onew[:,0],
            y=embedding_onew[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
import my_fSNE
weights = [1, 0.9,0.6]
results_fsne_slow = []

fSNE_slow = my_fSNE.my_fSNE(X_transformed, max_iter=5, POI = POI, lambdas = weights)

results_torch_fSNE = []
best_score = 1
for _ in range(1):
  ld_embedding_fSNE, p_matrix = fSNE_slow.fit_transform()
  print(p_matrix)
  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")
  if divergence < best_score:
    best_score = divergence
    np.save('fsne_w_1_09_06.npy', ld_embedding_fSNE, allow_pickle=True)
  results_fsne_slow.append([components[0], components[1], components[2], divergence])

ld_embedding_fSNE = np.load('fsne_w_1_09_06.npy', allow_pickle=True)
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
ld_embedding_fSNE = np.load('fsne_w_1_09_06.npy', allow_pickle=True)
data_instance = 26
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance, X_df.values[data_instance], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

print('Best Embedding')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
'''
import focusedSNE_torch
POI = [data_instance_number, 86]
#POI = []
weights = [2,1,0.75]
fSNE = focusedSNE_torch.TorchTSNE(verbose=True, n_iter=1, points_of_interest = POI, lambdas = weights)

results_torch_fSNE = []
for _ in range(1):
  ld_embedding_fSNE = fSNE.fit_transform(X_transformed)

  explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
  neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, ld_embedding_fSNE, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
  corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
  corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
  components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
  components = components.split(",")

  results_torch_fSNE.append([components[0], components[1], components[2], divergence])
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_fSNE[:,0],
            y=ld_embedding_fSNE[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)
'''

In [ ]:
results_old = np.array(results_orig_SNE, dtype=np.float32)
#results_old = np.load('results_old_50_500.npy', allow_pickle=True)
results_new = np.array(results_torch_SNE, dtype=np.float32)
#results_fnew = np.array(results_torch_fSNE, dtype=np.float32)
results_fslow = np.array(results_TorchSNE, dtype=np.float32)
mus_old = np.mean(results_old, axis = 0)
mus_new = np.mean(results_new, axis = 0)
#mus_fnew = np.mean(results_fnew, axis = 0)
mus_fnew = np.mean(results_fslow, axis = 0)
print(f'Average of {5} runs.')
print(f'POI = {POI}')
print(f'Discrepancy in Feature influence for: \nSNE_Python {mus_old[0]:.4f}, SNE_Torch {mus_new[0]:.4f}, fSNE with weights {1} {mus_fnew[0]:.4f}')
print(f'Discrepancy in Neighborhood Content for: \nSNE_Python {mus_old[1]:.4f}, SNE_Torch {mus_new[1]:.4f}, fSNE with weights {1} {mus_fnew[1]:.4f}')
print(f'Discrepancy in Feature Order for: \nSNE_Python {mus_old[2]:.4f}, SNE_Torch {mus_new[2]:.4f}, fSNE with weights {1} {mus_fnew[2]:.4f}')
print(f'Overall Divergence for: \nSNE_Python {mus_old[3]:.4f}, SNE_Torch {mus_new[3]:.4f}, fSNE with weights {1} {mus_fnew[3]:.4f}')

Average of 5 runs.
POI = [24, 86]
Discrepancy in Feature influence for: 
SNE_Python 0.7807, SNE_Torch 0.7807, fSNE with weights 1 0.7807
Discrepancy in Neighborhood Content for: 
SNE_Python 0.4500, SNE_Torch 0.4500, fSNE with weights 1 0.4500
Discrepancy in Feature Order for: 
SNE_Python 0.9000, SNE_Torch 0.9000, fSNE with weights 1 0.9000
Overall Divergence for: 
SNE_Python 0.7102, SNE_Torch 0.7102, fSNE with weights 1 0.7102


Average of 50 runs.

POI = [24, 86]

Discrepancy in Feature influence for: 

SNE_Python 0.7142, SNE_Torch 0.7749, fSNE with weights [1, 0.9, 0.6] 0.7169

Discrepancy in Neighborhood Content for: 

SNE_Python 0.4000, SNE_Torch 0.2500, fSNE with weights [1, 0.9, 0.6] 0.2650

Discrepancy in Feature Order for: 

SNE_Python 0.9500, SNE_Torch 0.8500, fSNE with weights [1, 0.9, 0.6] 0.8450

Overall Divergence for: 

SNE_Python 0.6881, SNE_Torch 0.6250, fSNE with weights [1, 0.9, 0.6] 0.6090

Average of 50 runs.

POI = [24, 86]

Discrepancy in Feature influence for: 

SNE_Python 0.7142, SNE_Torch 0.7749, fSNE with weights [1, 1, 1] 0.7183

Discrepancy in Neighborhood Content for: 

SNE_Python 0.4000, SNE_Torch 0.2500, fSNE with weights [1, 1, 1] 0.2500

Discrepancy in Feature Order for: 

SNE_Python 0.9500, SNE_Torch 0.8500, fSNE with weights [1, 1, 1] 0.9000

Overall Divergence for: 

SNE_Python 0.6881, SNE_Torch 0.6250, fSNE with weights [1, 1, 1] 0.6228

In [ ]:
#np.save('fSNE_equalW_interesting.npy', ld_embedding_fSNE,allow_pickle=True)
#ld_embedding_interesting = np.load('fSNE_equalW_interesting.npy', allow_pickle=True)
ld_embedding_interesting = ld_embedding_fSNE
explainer = LapsExplainer(X_df.values, feature_names=model_features, class_names=target, categorical_features=categorical_features, categorical_names=categorical_names, discretize_continuous=False,discretizer='quartile', random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(X_df.values, X_transformed, 
                                                                                                               ld_embedding_interesting, data_instance_number, X_df.values[data_instance_number], nbrs=20, num_features=5)
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors, oversampled_data, model_features, categorical_features, numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], neighbors_embd, oversampled_data_embd, model_features, categorical_features, numeric_features)
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")

#newlabels = labels
#for i in POI:
#  newlabels[i] = 2
#  y[i] = 2
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown', 2: 'Red'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant', 2: 'POI'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding_interesting[:,0],
            y=ld_embedding_interesting[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)
#print(f'Average of {all_runs} runs.')
print(f'POI = {POI}')
print(f'Weights = {weights}')
print(f'Discrepancy in Feature influence for: fSNE {components[0]}')
print(f'Discrepancy in Neighborhood Content for: fSNE {components[1]}')
print(f'Discrepancy in Feature Order for: fSNE with {components[2]}')
print(f'Overall Divergence for: \nfSNE with {divergence}')

In [ ]:
indexes = [i for i in range(0,len(y))]
colormap = {0: 'SteelBlue', 1: 'SandyBrown'}
colors = [colormap[z] for z in y.ravel()]
labels = {0: 'Benign', 1: 'Malignant'}
annotations = [labels[z] for z in y.ravel()]

source = ColumnDataSource(
        data=dict(
            x=ld_embedding[:,0],
            y=ld_embedding[:,1],
            all_colors = colors,
            label = annotations,
            indexes = indexes
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
        ]
    )

p = figure(plot_width=700, plot_height=450, tools=[hover],
           title="Embedding")
glyph = Text(x="x", y="y", text='indexes', x_offset=7, y_offset=7, text_font_size="9pt", text_color="grey")
p.circle('x', 'y', fill_color='all_colors', line_color='white', legend='label', size=10, source=source)
p.add_glyph(source, glyph)
show(p)

### Step 3: Obtain Representative Data-point and Select a Single Point

In [19]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [83, 27, 21, 4, 48, 18, 38, 37, 17, 51, 6, 26, 43, 41, 40, 80, 52, 58, 60, 66]
Points with Highest Density : [83, 27, 21, 4, 48, 18, 38, 37, 17, 51, 6, 26, 43, 41, 40, 80, 52, 58, 60, 66]
Points with Misplaced Neighborhoods : [69, 3, 49, 51, 12, 18, 90, 88, 75, 86, 14, 62, 64, 43, 40, 35, 58, 15, 82, 27]
Points Close to Decision Boundary : [[2, 0, 1], [2, 0, 1], [2, 1, 0], [2, 1, 0], [2, 1, 0], [2, 0, 1], [1, 2, 0], [1, 2, 0], [1, 2, 0], [2, 0, 1], [2, 0, 1], [2, 1, 0], [1, 2, 0], [2, 1, 0], [2, 1, 0], [2, 1, 0], [2, 0, 1], [1, 2, 0], [2, 1, 0], [2, 1, 0]]
Cluster Centres : [26, 70]


In [ ]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding_torch_SNE)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Highest Density : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Misplaced Neighborhoods : [44, 27, 48, 16, 89, 24, 55, 40, 2, 62, 84, 13, 83, 72, 74, 23, 29, 46, 25, 28]
Points Close to Decision Boundary : [88, 33, 39, 62, 18, 97, 17, 29, 16, 47, 46, 43, 2, 21, 48, 6, 75, 69, 8, 44]
Cluster Centres : [24, 86]


In [ ]:
rep_points = genericMethods.generate_representative_points(20, X_df, X_transformed, y, ld_embedding_fSNE)
Rep_order = ["Density Based Outliers", "Points with Highest Density", "Points with Misplaced Neighborhoods", "Points Close to Decision Boundary", "Cluster Centres"]

for index in range(0,len(rep_points)):
    print(Rep_order[index]+" : "+''.join(str(rep_points[index])))

Density Based Outliers : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Highest Density : [89, 36, 40, 62, 52, 15, 6, 56, 83, 2, 80, 53, 48, 47, 46, 44, 7, 29, 27, 81]
Points with Misplaced Neighborhoods : [44, 48, 16, 27, 89, 24, 55, 62, 2, 40, 84, 6, 25, 36, 72, 74, 29, 28, 7, 13]
Points Close to Decision Boundary : [88, 33, 39, 62, 18, 97, 17, 29, 16, 47, 46, 43, 2, 21, 48, 6, 75, 69, 8, 44]
Cluster Centres : [24, 86]


In [ ]:
## user selects an instance

data_instance_number = 12

### Step 4: Initiate and Execute LAPS Explainer

#### a. Initiate Explainer

In [20]:
explainer = LapsExplainer(X_df.values,
                feature_names=model_features,
                class_names=target,
                categorical_features=categorical_features,
                categorical_names=categorical_names,
                discretize_continuous=False,
                discretizer='quartile',
                random_state=42)
neighbors, neighbors_embd, oversampled_data, oversampled_data_embd = explainer.generate_perturbed_neighborhood(
                                                                                X_df.values,
                                                                                X_transformed,
                                                                                ld_embedding,
                                                                                data_instance_number,
                                                                                X_df.values[data_instance_number],
                                                                                nbrs=20,
                                                                                num_features=5)

#### b. Explain the Point in Original Data and in Embedding

In [23]:
corr_feat_dist, feature_dict, feature_distance_contribution, dvs_matrix, sorted_indexes = Explanation.explain_point_local(X_df.values[data_instance_number], 
                                                                                                              neighbors, 
                                                                                                              oversampled_data, 
                                                                                                              model_features, 
                                                                                                              categorical_features, 
                                                                                                              numeric_features)
corr_feat_dist_embd, feature_dict_embd, feature_distance_contribution_embd, dvs_matrix_embd, sorted_indexes_embd = Explanation.explain_point_local(X_df.values[data_instance_number], 
                                                                                                                                       neighbors_embd, 
                                                                                                                                       oversampled_data_embd, 
                                                                                                                                       model_features, 
                                                                                                                                       categorical_features, 
                                                                                                                                       numeric_features)

#### c. Plot Feature Contribution in Original Data and in Embedding

In [24]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict.keys()),
            y_val = list(feature_dict.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict.keys()), plot_height=380, plot_width=980, toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-15, y_offset='y_offset', text_font_size="9pt", text_font_style = 'bold', text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.7, source=source)
p.xaxis.major_label_orientation = math.pi/7
p.xaxis.major_label_text_font_size = '9pt'
p.yaxis.major_label_text_font_size = '9pt'
p.xaxis.major_label_text_font_style = 'bold'
p.yaxis.major_label_text_font_style = 'bold'
p.min_border_left = 82
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_align = 'center'
p.y_range.start = np.min(list(feature_dict.values()))-0.05
p.y_range.end = np.max(list(feature_dict.values()))+0.01
p.add_glyph(source, glyph)
#export_png(p, filename="Images/Fig4/Feature-Influences-85-Original.png")
show(p)

In [25]:
bar_color = []
y_offset_val = []
label = []
for item in feature_dict_embd.values():
    label.append("{0:.2f}".format(item))
    if item<0:
        bar_color.append('#e34a33')
        y_offset_val.append(item+15)
    else:
        bar_color.append('#2ca25f')
        y_offset_val.append(item)
        
        
source = ColumnDataSource(
        data=dict(
            x_val = list(feature_dict_embd.keys()),
            y_val = list(feature_dict_embd.values()),
            y_offset = y_offset_val,
            labels =  label,
            color = bar_color
        )
    )

p = figure(x_range=list(feature_dict_embd.keys()), plot_height=380, plot_width=980, toolbar_location=None, tools="")

glyph = Text(x="x_val", y="y_val", text='labels', x_offset=-15, y_offset='y_offset', text_font_size="9pt", text_font_style = 'bold', text_color="black")


p.vbar(x="x_val", top="y_val", color="color", width=0.7, source=source)
p.xaxis.major_label_orientation = math.pi/7
p.xaxis.major_label_text_font_size = '9pt'
p.yaxis.major_label_text_font_size = '9pt'
p.xaxis.major_label_text_font_style = 'bold'
p.yaxis.major_label_text_font_style = 'bold'
p.min_border_left = 80
p.xgrid.grid_line_color = None
p.xaxis.axis_label_text_align = 'center'
p.y_range.start = np.min(list(feature_dict.values()))-0.05
p.y_range.end = np.max(list(feature_dict.values()))+0.01
p.add_glyph(source, glyph)
#export_png(p, filename="Images/Fig4/Feature-Influences-85-Embedding.png")
show(p)

#### d. Calculate Local Divergence

In [26]:
components, divergence = Explanation.compute_local_divergence(corr_feat_dist, corr_feat_dist_embd, neighbors, neighbors_embd)
components = components.split(",")

print("Discrepancy in Feature Influence:", components[0])
print("Discrepancy in Neighborhood Content:", components[1])
print("Discrepancy in Neighborhood Order:", components[2])
print("Overall Divergence: ", divergence)

Discrepancy in Feature Influence: 0.8262592742384838
Discrepancy in Neighborhood Content: 0.15
Discrepancy in Neighborhood Order: 0.8
Overall Divergence:  0.5920864247461612


Discrepancy in Feature Influence: 0.7374545801730664

Discrepancy in Neighborhood Content: 0.35

Discrepancy in Neighborhood Order: 0.9

Overall Divergence:  0.6624848600576887